In [1]:
import tensorflow as tf
import keras 
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.regularizers import l2
from keras import callbacks

2022-07-30 15:45:39.156336: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-07-30 15:45:40.808695: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-30 15:45:40.879609: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 15:45:40.903027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 15:45:40.903111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so ret

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8267722960145874090
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 23213703168
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12722680529640039208
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

ysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 15:45:41.382310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 15:45:41.382377: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /device:GPU:0 with 22138 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6


In [3]:
output_classes = 2
image_height, image_width = 580, 400
image_shape = (image_height, image_width)
epochs = 10;


In [4]:
train_data = r'/home/cybertech1/data/filtered_balanced/train_set'
val_data=r'/home/cybertech1/data/filtered_balanced/validation_set'
test_data = r'/home/cybertech1/data/filtered_balanced/test_set'

In [5]:
#if k.image_data_format() == 'channels_first':
#  input_shape = (3, img_width, img_height)
#else:
#  input_shape = (img_width, img_height, 3)

In [6]:
print('train: ')
for root,dirs,files in os.walk(train_data):
    print (root, len(files))
print('val: ')
for root,dirs,files in os.walk(val_data):
    print (root, len(files))
print('test: ')
for root,dirs,files in os.walk(test_data):
    print (root, len(files))

train: 
/home/cybertech1/data/filtered_balanced/train_set 0
/home/cybertech1/data/filtered_balanced/train_set/NONE 0
/home/cybertech1/data/filtered_balanced/train_set/NONE/NONE 3718
/home/cybertech1/data/filtered_balanced/train_set/t 0
/home/cybertech1/data/filtered_balanced/train_set/t/t 1859
val: 
/home/cybertech1/data/filtered_balanced/validation_set 0
/home/cybertech1/data/filtered_balanced/validation_set/NONE 0
/home/cybertech1/data/filtered_balanced/validation_set/NONE/NONE 2222
/home/cybertech1/data/filtered_balanced/validation_set/t 0
/home/cybertech1/data/filtered_balanced/validation_set/t/t 1111
test: 
/home/cybertech1/data/filtered_balanced/test_set 0
/home/cybertech1/data/filtered_balanced/test_set/NONE 0
/home/cybertech1/data/filtered_balanced/test_set/NONE/NONE 13379
/home/cybertech1/data/filtered_balanced/test_set/t 0
/home/cybertech1/data/filtered_balanced/test_set/t/t 1668


In [7]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip = True,
)

train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size = (image_shape),
    batch_size = 32,
    class_mode = 'binary'
)

val_generator = train_datagen.flow_from_directory(
    val_data,
    target_size = (image_shape),
    batch_size = 32,
    class_mode = 'binary'
)

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
    test_data,
    target_size = (image_shape),
    batch_size = 32,
    class_mode = 'binary'
)


Found 5577 images belonging to 2 classes.
Found 3333 images belonging to 2 classes.
Found 15047 images belonging to 2 classes.


In [8]:
base_model = tf.keras.applications.DenseNet121(
    weights='imagenet', 
    include_top=False, 
    input_tensor=Input(shape=(image_shape+(3,)))
    )
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
prediction = Dense(output_classes, activation=tf.nn.softmax)(x)

model = Model(inputs=base_model.input,outputs=prediction)

2022-07-30 15:46:15.448504: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 15:46:15.448664: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 15:46:15.448751: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 15:46:15.448988: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 15:46:15.449086: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [9]:
'''
model.compile(optimizer= 'adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])
'''
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [10]:
#przerywamy jeśli wystepuje overfitting
tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

train_steps = 5577//32
val_steps = 5577//32


history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs = 32,
    validation_data=val_generator,
    validation_steps=12
)

Epoch 1/32


2022-07-30 15:46:30.558359: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8302
2022-07-30 15:46:32.542816: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


174/174 [==============================] - 128s 679ms/step - loss: 0.8146 - accuracy: 0.6534 - val_loss: 0.8038 - val_accuracy: 0.6432
Epoch 2/32
174/174 [==============================] - 115s 656ms/step - loss: 0.7736 - accuracy: 0.6647 - val_loss: 0.7673 - val_accuracy: 0.6589
Epoch 3/32
174/174 [==============================] - 113s 648ms/step - loss: 0.7452 - accuracy: 0.6660 - val_loss: 0.7063 - val_accuracy: 0.6979
Epoch 4/32
174/174 [==============================] - 113s 646ms/step - loss: 0.7263 - accuracy: 0.6653 - val_loss: 0.6982 - val_accuracy: 0.7031
Epoch 5/32
174/174 [==============================] - 115s 659ms/step - loss: 0.6996 - accuracy: 0.6678 - val_loss: 0.7824 - val_accuracy: 0.4974
Epoch 6/32
174/174 [==============================] - 113s 650ms/step - loss: 0.6835 - accuracy: 0.6774 - val_loss: 0.7003 - val_accuracy: 0.6458
Epoch 7/32
174/174 [==============================] - 113s 648ms/step - loss: 0.6633 - accuracy: 0.6862 - val_loss: 0.6823 - val_accura

In [11]:
model.evaluate(test_generator)

471/471 [==============================] - 48s 101ms/step - loss: 0.6951 - accuracy: 0.6216


[0.6951245665550232, 0.6215857267379761]